In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('./cl1.csv')

In [14]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
                df.drop('diagnosis', axis=1),
                df['diagnosis'],
                test_size=0.2,
                random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)

Shape of training set: (116, 31)
Shape of test set: (29, 31)


In [15]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions1 = logreg.predict(X_test)

c:\Users\fafaf\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))
print('\n')
print(classification_report(y_test, predictions1))

Confusion Matrix: 
 [[14  0]
 [ 0 15]]


              precision    recall  f1-score   support

           B       1.00      1.00      1.00        14
           M       1.00      1.00      1.00        15

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



In [20]:
predictions2 = logreg.predict(X_train)

print("Confusion Matrix: \n", confusion_matrix(y_train, predictions2))
print('\n')
print(classification_report(y_train, predictions2))

Confusion Matrix: 
 [[46  3]
 [ 2 65]]


              precision    recall  f1-score   support

           B       0.96      0.94      0.95        49
           M       0.96      0.97      0.96        67

    accuracy                           0.96       116
   macro avg       0.96      0.95      0.96       116
weighted avg       0.96      0.96      0.96       116



In [6]:
model = LogisticRegression(warm_start=True,penalty='l2')

In [7]:
import utils
utils.set_initial_params(model, n_features=X_train.shape[1], n_classes=2)

In [8]:
import flwr as fl
import warnings
from sklearn.metrics import log_loss

class IrisClient(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return utils.get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_train, y_train)
            accuracy = model.score(X_train, y_train)
            return (
                utils.get_model_parameters(model),
                len(X_train),
                {"train_accuracy": accuracy},
            )

        def evaluate(self, parameters, config):  # type: ignore
            utils.set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            return loss, len(X_test), {"test_accuracy": accuracy}

    # Start Flower client
fl.client.start_client(
    server_address="127.0.0.1:8080", client=IrisClient().to_client()
)

INFO flwr 2024-05-01 23:29:24,167 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-05-01 23:29:24,169 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-05-01 23:29:24,171 | connection.py:55 | ChannelConnectivity.READY
DEBUG flwr 2024-05-01 23:29:27,840 | connection.py:220 | gRPC channel closed
INFO flwr 2024-05-01 23:29:27,840 | app.py:398 | Disconnect and shut down


In [ ]:
print(model.coef_)

In [10]:
predictions1 = model.predict(X_test)
print(classification_report(y_test, predictions1))

              precision    recall  f1-score   support

           B       0.93      1.00      0.97        14
           M       1.00      0.93      0.97        15

    accuracy                           0.97        29
   macro avg       0.97      0.97      0.97        29
weighted avg       0.97      0.97      0.97        29



In [11]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))

Confusion Matrix: 
 [[14  0]
 [ 1 14]]


In [21]:
predictions1 = model.predict(X_train)
print(classification_report(y_train, predictions1))

              precision    recall  f1-score   support

           B       0.96      0.96      0.96        49
           M       0.97      0.97      0.97        67

    accuracy                           0.97       116
   macro avg       0.96      0.96      0.96       116
weighted avg       0.97      0.97      0.97       116



In [22]:
print("Confusion Matrix: \n", confusion_matrix(y_train, predictions1))

Confusion Matrix: 
 [[47  2]
 [ 2 65]]
